In [1]:
import plotly.plotly as py
import plotly.graph_objs as go
import pandas as pd
import plotly
import numpy as np
import random

plotly.offline.init_notebook_mode(connected=True)

In [2]:
np.random.seed(1234)
real = np.random.choice(np.array([-1,1]),10000)
probas=np.vectorize(lambda x:random.normalvariate(.25*x+0.5,0.3))
predicted=probas(real)
predicted = np.where(predicted>1,1,np.where(predicted<0,0,predicted))
digitsRound=3
positive=1

In [3]:
def tablaMedidas(real,predicted,positive,digitsRound=3):
    ds = pd.DataFrame({'predicted':np.round(predicted,digitsRound),'real':np.where(real==positive,'P','N')}
                  ).pivot_table(index=['predicted'],
                                columns=['real'],
               aggfunc=len).fillna(0).reset_index().sort_values(by=['predicted'],ascending=False)

    ds['Tot']=ds.P+ds.N

    sumP=sum(ds.P)
    sumN=sum(ds.N)
    sumTot=sum(ds.Tot)

    ds['PAcum']=np.cumsum(ds.P)
    ds['NAcum']=np.cumsum(ds.N)
    ds['TotAcum']=np.cumsum(ds.Tot)
    ds['Sens']=ds.PAcum/sumP
    ds['UnoMSpec']=ds.NAcum/sumN
    ds['Precision']=ds.PAcum/ds.TotAcum
    ds['Accuracy']=(ds.PAcum+sumN-ds.NAcum)/sumTot
    ds['F1Score']=2*(ds.Sens*ds.Precision)/(ds.Sens+ds.Precision)
    ds['PropTot']=ds.TotAcum/sumTot
    ds['Lift']=ds.Precision/(sumP/sumTot)
    
    return ds


    


In [4]:
def ROCCA(real,predicted,positive,digitsRound=3):
    
    ds = tablaMedidas(real,predicted,positive,digitsRound)
    
    AUC = sum(((ds.Sens  + np.append(0,ds.Sens[:-1]))/2)*((ds.UnoMSpec-np.append(0,ds.UnoMSpec[:-1]))))
    
    dsVacio = pd.DataFrame({'predicted':np.nan,
                        'N':0,
                        'P':0,
                        'Tot':0,
                        'PAcum':0,
                        'NAcum':0,
                        'TotAcum':0,
                        'Sens':0,
                        'UnoMSpec':0,
                        'Precision':np.nan,
                        'Accuracy':0,
                        'F1Score':np.nan,
                        'PropTot':0,
                        'Lift':np.nan}
                       ,index=[0])

    ds=pd.concat([dsVacio,ds])

    trace0 = go.Scatter(
            x = ds.UnoMSpec,
            y= ds.Sens,
            mode = 'lines',
            text = 'Sensibilidad: ' + np.round(ds.Sens,digitsRound).astype(str) + '<br>\
    1-Especificidad: ' + np.round(ds.UnoMSpec,digitsRound).astype(str) + '<br>\
    Probabilidad: ' + ds.predicted.astype(str) + '<br>\
    Población: ' + ds.TotAcum.astype(int).astype(str) + ' (' + (np.round(ds.PropTot,2)*100).astype(int).astype(str) + '%)' + '<br>\
    Casos Positivos: ' + ds.PAcum.astype(int).astype(str) + '<br>\
    Casos Negativos: ' + ds.NAcum.astype(int).astype(str) + '<br>\
    Accuracy: ' + np.round(ds.Accuracy,digitsRound).astype(str) + '<br>\
    Precisión: ' + np.round(ds.Precision,digitsRound).astype(str) + '<br>\
    F1-Score: ' + np.round(ds.F1Score,digitsRound).astype(str) + '<br>\
    Lift: ' + np.round(ds.Lift,digitsRound).astype(str),
            hoverinfo = 'text',
            showlegend=False
            )

    trace1 = go.Scatter(x=[0,1],
                       y=[0,1],
                       mode='lines',
                       hoverinfo='none',
                       showlegend=False)

    layout= go.Layout(yaxis=dict(
            title='Sensibilidad'
            ),
                     xaxis=dict(title='1-Especificidad'),
                     annotations=[
        dict(
            x=0.8,
            y=0.2,
            showarrow = False,
            text = 'AUC: ' + str(AUC))
    ])

    plotly.offline.iplot(go.Figure(data=[trace0,trace1],layout=layout),filename='archivo')

In [5]:
ROCCA(real,predicted,1)